# Cyclus parameter fidelity assessment tool

This tool is designed to help users understand the behavior of Cyclus archetypes.

In [1]:
import subprocess

In [2]:
#import cymetric as cym
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [ ]:
plt.rcParams['figure.figsize'] = (40,24)

In [ ]:
from cymetric import graphs as cgr
from cymetric import timeseries as tm
from cymetric import filters as fl

In [ ]:
import cyclus

In [1]:
print(cyclus.__file__)
cyclus.lib.

SyntaxError: invalid syntax (<ipython-input-1-9868dd3fe39d>, line 2)

Enter two file paths to compare: 
(expand this later?)

In [1]:
infile1 = "candu_one_day.py"
infile2 = "candu_one_month.py"

In [2]:
process1 = subprocess.run(['cyclus', infile1, '-o', 'candu_one_day2.sqlite'], stdout=subprocess.PIPE)

-m cyclus "candu_one_day.py"
# with open('candu_one_day.log', 'w') as f:
#     print(process1.stdout.decode('utf-8'))
#     f.write(process1.stdout.decode('utf-8'))

SyntaxError: invalid syntax (<ipython-input-2-ac9d9921343e>, line 3)

In [ ]:
#process2 = subprocess.run(['cyclus', infile2, '-o', 'candu_one_month.sqlite'], stdout=subprocess.PIPE)

# with open('candu_one_month.log', 'w') as f:
#     print(process2.stdout.decode('utf-8'))
#     f.write(process2.stdout.decode('utf-8'))

In [ ]:
outfile1 = "candu_one_day.sqlite"
db = cym.dbopen(outfile1)
ev = cym.Evaluator(db=db, write=True)

In [ ]:
cym.graphs.flow_graph(ev, label = "mass")

In [ ]:
outfile2 = "candu_one_month.sqlite"
db2 = cym.dbopen(outfile2)
ev2 = cym.Evaluator(db=db2, write=True)

In [ ]:
cym.graphs.flow_graph(ev2, label = "mass")

In [ ]:
outfile3 = "candu_unscaled.sqlite"
db3 = cym.dbopen(outfile3)
ev3 = cym.Evaluator(db=db3, write=True)

In [ ]:
cym.graphs.flow_graph(ev3, label = "mass")

In [ ]:
day_quantities = []
for i in range(4320): 
    subset = ev.eval("ExplicitInventory", conds = [("Time", "==", i)])
    day_quantities.append(sum(subset["Quantity"]))

month_quantities = []
for i in range(144):
    subset = ev2.eval("ExplicitInventory", conds = [("Time", "==", i)])
    month_quantities.append(sum(subset["Quantity"]))
    
unscaled_quantities = []
for i in range(144):
    subset = ev3.eval("ExplicitInventory", conds = [("Time", "==", i)])
    unscaled_quantities.append(sum(subset["Quantity"]))

In [ ]:
fig, ax1 = plt.subplots(figsize = (10,8))

ax1.plot(range(144), month_quantities, color = "b", linewidth = 2.0, label = "dt = one month")
ax1.plot(range(144), unscaled_quantities, color = "y", linewidth = 2.0, label = "cycle time = 17")
ax1.set_xlabel("time (months)")
ax1.set_ylabel("material (kg)")
ax1.legend(loc = "upper left")

ax2 = ax1.twiny()
ax2.plot(range(4320), day_quantities, color = "r", linewidth = 1.0, label = "dt = one day")
ax2.set_xlabel("time (days)")

plt.title("Cumulative Cyclus simulation inventory")
plt.legend(loc='lower right') 

plt.show()